In [1]:
import os
import nengo
import nengo_dl
import numpy as np
import keras
import sklearn.model_selection
import tensorflow as tf
import snntoolbox.bin.run
from snntoolbox.utils.utils import import_configparser
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Dropout, AveragePooling2D, Flatten, Dense

In [2]:
os.makedirs('params', exist_ok=True)
dataset_path = os.path.join('..', 'datasets', 'output', 'p300-target-nontarget.npz')
nengo_keras_params_path = os.path.join('params', 'nengo_keras_params')
model_name = 'keras_snn_toolbox'
snn_toolbox_model_output = os.path.join('params', model_name)
snn_toolbox_config_path = os.path.join('params', 'snn_config')

In [3]:
def get_dataset(file):
    dataset = np.load(file)
    target_x, non_target_x = dataset['target_features'], dataset['non_target_features']

    if target_x.shape[0] > non_target_x.shape[0]:
        target_x = target_x[:non_target_x.shape[0]]
    else:
        non_target_x = non_target_x[:target_x.shape[0]]

    # (1,0) for target data, (0,1) for non target data
    target_y = np.tile(np.array([1, 0]), (target_x.shape[0], 1))
    non_target_y = np.tile(np.array([0, 1]), (non_target_x.shape[0], 1))

    features = np.concatenate((target_x, non_target_x), axis=0)
    labels = np.concatenate((target_y, non_target_y), axis=0)

    features = np.expand_dims(features, -1)

    return features, labels

In [4]:
def create_nn():
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(3, 3), activation=keras.activations.elu, input_shape=(3, 1201, 1),
                     name='input_layer'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5, seed=0))
    model.add(AveragePooling2D(pool_size=(1, 8)))
    model.add(Flatten())
    model.add(Dense(100, activation=keras.activations.elu))
    model.add(BatchNormalization())
    model.add(Dropout(0.5, seed=0))
    model.add(Dense(2, activation=keras.activations.softmax, name='output_layer'))
    model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

    return model

In [5]:
def run_vanilla_tensorflow(train_x, train_y, valid_x, valid_y, test_x, test_y, test_results):
    vanilla_tensorflow_model = create_nn()
    vanilla_tensorflow_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    vanilla_tensorflow_model.fit(
        x=train_x,
        y=train_y,
        shuffle=True,
        epochs=30,
        batch_size=16,
        callbacks=[keras.callbacks.EarlyStopping(patience=5, verbose=1, restore_best_weights=1)],
        validation_data=(valid_x, valid_y)
    )

    eval = vanilla_tensorflow_model.evaluate(test_x, test_y)

    test_results.append(eval)
    return vanilla_tensorflow_model


In [6]:
def run_spiking_network(activation, model: tf.keras.Model, ann_params, snn_results, test_x, test_y, scale_firing_rates=1,
                        synapse=None, n_steps=30):
    converter = nengo_dl.Converter(
        model,
        swap_activations={tf.nn.elu: activation},
        scale_firing_rates=scale_firing_rates,
        synapse=synapse
    )

    input_layer, output_layer = model.get_layer('input_layer'), model.get_layer('output_layer')
    nengo_input, nengo_output = converter.inputs[input_layer], converter.outputs[output_layer]

    tiled_data = np.tile(test_x, (1, n_steps, 1))

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    with nengo_dl.Simulator(converter.net, minibatch_size=16, progress_bar=False) as sim:
        sim.load_params(ann_params)
        data = sim.predict(tiled_data)

    predictions = np.argmax(data[nengo_output][:, -1], axis=-1)
    accuracy = (predictions == test_y).mean()
    print('accuracy: {:4f}%'.format(accuracy*100))
    snn_results.append(accuracy)

In [7]:
def run_nengo_model(train_x, train_y, valid_x, valid_y, test_x, test_y, test_results, snn_results):
    model = create_nn()
    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=16) as sim:
        sim.compile(
            optimizer=tf.optimizers.Adam(),
            loss=tf.losses.BinaryCrossentropy(),
            metrics=['accuracy']
        )

        sim.fit(
            x=train_x,
            y=train_y,
            shuffle=True,
            epochs=30,
            callbacks=[keras.callbacks.EarlyStopping(patience=5, verbose=1, restore_best_weights=1)],
            validation_data=(valid_x, valid_y)
        )

        eval = sim.evaluate(test_x, test_y)

        test_results.append(eval)

        sim.save_params(nengo_keras_params_path)

    run_spiking_network(
        activation=nengo.SpikingRectifiedLinear(),
        model=model,
        ann_params=nengo_keras_params_path,
        test_x=test_x,
        test_y=test_y,
        synapse=0.01,
        snn_results=snn_results
    )

In [8]:
def run_snn_toolbox(snn_results, config_file_path):
    return snn_results.append(snntoolbox.bin.run.main(config_file_path))

def snn_config(model_path, dataset_path, model_name, test_count):
    config_parser = import_configparser()
    config = config_parser.ConfigParser()

    config['paths'] = {
        'path_wd': model_path,
        'dataset_path': dataset_path,
        'filename_ann': model_name
    }

    config['tools'] = {
      'evaluate_ann': False,
      'normalize': True
  }

    config['simulation'] = {
        'simulator': 'INI',
        'duration': 50,
        'num_to_test': test_count,
        'batch_size': 10
    }
    return config

In [9]:
print(tf.config.list_physical_devices('GPU'))

seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

features, labels = get_dataset(dataset_path)
print(features.shape, labels.shape)

test_results, snn_results = [], []

train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(
    features, labels, test_size=0.25, random_state=seed, shuffle=True
)

np.savez_compressed(os.path.join('params','x_test'), test_x)
np.savez_compressed(os.path.join('params','y_test'), test_y)
np.savez_compressed(os.path.join('params','x_norm'), train_x[::10])

config = snn_config(
    model_path=os.path.join('params'),
    dataset_path=os.path.join('params'),
    model_name=model_name,
    test_count=test_x.shape[0]
)
with open(snn_toolbox_config_path, 'w') as file:
    config.write(file)

shuffle_split = sklearn.model_selection.ShuffleSplit(n_splits=5, test_size=0.25, random_state=seed)

for train, valid in shuffle_split.split(train_x):
    current_train_x = train_x[train]
    current_train_y = train_y[train]
    current_valid_x = train_x[valid]
    current_valid_y = train_y[valid]

    # run_nengo_model(
    #     train_x=current_train_x,
    #     train_y=current_train_y,
    #     valid_x=current_train_x,
    #     valid_y=current_valid_y,
    #     test_x=test_x,
    #     test_y=test_y,
    #     test_results=test_results,
    #     snn_results=snn_results
    # )

    model = run_vanilla_tensorflow(
        train_x=current_train_x,
        train_y=current_train_y,
        valid_x=current_valid_x,
        valid_y=current_valid_y,
        test_x=test_x,
        test_y=test_y,
        test_results=test_results
    )
    model.save(snn_toolbox_model_output + '.h5')

    run_snn_toolbox(snn_results, config_file_path=snn_toolbox_config_path)


for i in range(len(test_results)):
    print('ANN acc: {}, SNN acc: {}'.format(test_results[i][1], snn_results[i]))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
(3050, 3, 1201, 1) (3050, 2)
Epoch 1/30
108/108 [==============================] - 1s 8ms/step - loss: 0.6709 - accuracy: 0.5936 - val_loss: 0.6916 - val_accuracy: 0.5262

IndexError: list index out of range

In [10]:
print(test_results, snn_results)

[[0.6984658241271973, 0.45478373765945435], [0.6885229349136353, 0.5452162623405457], [0.7408103346824646, 0.45478373765945435], [0.7071878910064697, 0.45478373765945435], [0.7084885835647583, 0.45478373765945435]] []
